In [3]:
from flask import Flask, request, jsonify
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import time

# ====== LOAD MODEL & DATA LÊN RAM KHI SERVER START ======
app = Flask(__name__)

# Giả sử bạn đã có file "indexed_data.pkl" chứa:
#   [
#       {
#           "header_index": ...,
#           "header": ...,
#           "text": ...,
#           "embedding": np.array(...)
#       },
#       ...
#   ]
with open("indexed_data.pkl", "rb") as f:
    indexed_data = pickle.load(f)

# Tạo list embeddings
content_embeddings_array = np.vstack([item["embedding"] for item in indexed_data])
headers = [item["header"] for item in indexed_data]
texts = [item["text"] for item in indexed_data]

model = SentenceTransformer("all-MiniLM-L6-v2")


def find_best_chunks(question, content_weight=1.0, k=2):
    """
    Lấy top-k đoạn văn phù hợp nhất từ 'indexed_data' dựa trên câu hỏi.
    """

    # Embed câu hỏi
    question_embedding = model.encode([question])

    # Tính độ tương đồng với mỗi chunk
    content_similarities = cosine_similarity(question_embedding, content_embeddings_array)[0]

    # Lấy top-k
    sorted_indices = np.argsort(content_similarities)[::-1]  # descending
    top_indices = sorted_indices[:k]

    top_chunks = [indexed_data[i] for i in top_indices]
    return top_chunks

def generate_answer(question, chunks):
    """
    Tạm minh họa: ghép context + question rồi trả về 1 string làm câu trả lời.
    Có thể thay bằng LLM (Gemini/OpenAI/…).
    """
    # Ghép tất cả text trong chunks làm context
    context = "\n\n---\n\n".join([chunk["text"] for chunk in chunks])
    
    # Ví dụ logic đơn giản: Trả về (context + " => answer"). 
    # Thực tế bạn dùng LLM (Gemini, OpenAI) thay thế tùy ý.
    final_answer = f"[Context]\n{context}\n\n[User Question]\n{question}\n\n[Answer]\nThis is a mock answer from local Python server."
    return final_answer


# ====== TẠO ENDPOINT CHÍNH CHO FRONTEND GỌI ======
@app.route("/ask", methods=["POST"])
def ask_chatbot():
    data = request.json
    question = data.get("question", "")

    # Logic truy xuất
    top_chunks = find_best_chunks(question, k=2)
    answer_text = generate_answer(question, top_chunks)

    return jsonify({"answer": answer_text})

# ====== CHẠY SERVER ======
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.16.23.4:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1